In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import random
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt

In [1]:
# images_dir is a dataset directory
images_dir = 
faces_dir = 
multi_dir = 

In [ ]:
# Destination directory with 200x200 images
if not os.path.exists(faces_dir):
   os.makedirs(faces_dir)

In [ ]:
# Destination directory with multi-face images
if not os.path.exists(multi_dir):
   os.makedirs(multi_dir)

## Cutting images to 200x200

In [ ]:
modelFile = "../../Models/face-detection/res10_300x300_ssd_iter_140000_fp16.caffemodel"
configFile = "../../Models/face-detection/deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

In [ ]:
def cut_images(path, newpath):
    if os.path.exists(newpath):
        return

    f = cv2.imread(path, 1)
    blob = cv2.dnn.blobFromImage(cv2.resize(f, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_count = 0
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            face_count += 1

    if face_count > 1:
        img_folder = os.path.join(multi_dir, os.path.splitext(os.path.basename(path))[0])
        os.makedirs(img_folder, exist_ok=True)

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([f.shape[1], f.shape[0], f.shape[1], f.shape[0]])
                (x, y, x2, y2) = box.astype("int")
                r = (x2-x)-(y2-y)
                if(r>0):
                    m = (y+y2)//2
                    y = m - (x2-x)//2
                    y2 = m + (x2-x)//2
                elif(r<0):
                    m = (x+x2)//2
                    x= m - (y2-y)//2
                    x2 = m + (y2-y)//2
                if(x<0 or y<0):
                    continue
                extracted_face = f[y:y2, x:x2]
                height, width, channels = extracted_face.shape
                if(max(width, height)<200):
                    continue
                extracted_face = cv2.resize(extracted_face, (200,200))
                cv2.imwrite(os.path.join(img_folder, f'face_{i}.jpg'), extracted_face)
    else:
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * \
                    np.array([f.shape[1], f.shape[0], f.shape[1], f.shape[0]])
                (x, y, x2, y2) = box.astype("int")
                r = (x2-x)-(y2-y)
                if(r > 0):
                    m = (y+y2)//2
                    y = m - (x2-x)//2
                    y2 = m + (x2-x)//2
                elif(r < 0):
                    m = (x+x2)//2
                    x = m - (y2-y)//2
                    x2 = m + (y2-y)//2
                if(x < 0 or y < 0):
                    continue
                extracted_face = f[y:y2, x:x2]
                height, width, channels = extracted_face.shape
                if(max(width, height) < 200):
                    continue
                extracted_face = cv2.resize(extracted_face, (200, 200))
                cv2.imwrite(newpath, extracted_face)
[cut_images(os.path.join(images_dir, filename), os.path.join(faces_dir, filename)) for filename in os.listdir(images_dir)]